In [308]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator

import pymongo

In [229]:
load_dotenv()

True

#### Paris: Fontain Agam. Melbourne: Australia Post. Sao Paulo: Sala Sao Paulo

In [230]:
paris_ld = {'type': 'Point', 'coordinates': ['48.8905568192578','2.242512906913018']}

In [250]:
melbourne_cbd = {'type': 'Point', 'coordinates': ['-37.81227175879301','144.96262186884945']}

In [19]:
sao_paulo_zc = {'type': 'Point', 'coordinates': ['-23.533215960575276','-46.639408758375076']}

In [231]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    #v = f'{venue}'
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [232]:
datap = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'vegan')

In [281]:
datam = getdata_fsq('https://api.foursquare.com/v2/venues/search', melbourne_cbd, 'airport')

In [282]:
datam_= datam['response']['venues']

In [283]:
datam_[0]

{'id': '4bd85e8a2ecdce72b5c1d0f2',
 'name': 'Airport Bed & Breakfast',
 'contact': {},
 'location': {'address': '314 Melrose Dr.',
  'lat': -37.81541246757951,
  'lng': 144.9830547407441,
  'labeledLatLngs': [{'label': 'display',
    'lat': -37.81541246757951,
    'lng': 144.9830547407441}],
  'distance': 1830,
  'postalCode': '3043',
  'cc': 'AU',
  'city': 'Tullamarine',
  'state': 'VIC',
  'country': 'Australia',
  'formattedAddress': ['314 Melrose Dr.',
   'Tullamarine VIC 3043',
   'Australia']},
 'categories': [{'id': '4bf58dd8d48988d1f8931735',
   'name': 'Bed & Breakfast',
   'pluralName': 'Bed & Breakfasts',
   'shortName': 'B & B',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/bedandbreakfast_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'hereNow

In [233]:
datap_= datap['response']['venues']

In [284]:
map_name = ['name']
#map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']

In [285]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

In [286]:
def type_point(gps):
    return {"type": "Point", "coordinates": gps}

#### HACER FUNCION PARA GENERAR UN JSON POR CADA DATA agregar Tipo (vegan, starb, etc)

In [295]:
gastr = []
for dic in datap_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

In [296]:
airp = []
for dic in datam_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
#    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["longitude"], ndic["latitude"]])
    airp.append(ndic)

#### HACER FUNCION PARA GENERAR LOS JSON

In [239]:
#def jsongenerator(name):
with open('vegans.json', 'w') as f:
    json.dump(gastr, f)

In [297]:
with open('airportsm.json', 'w') as f:
    json.dump(airp, f)

In [309]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [310]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [300]:
from pymongo import GEOSPHERE

In [311]:
g = db.get_collection("airport")

In [312]:
g.find_one().keys()

dict_keys(['_id', 'name', 'latitude', 'longitude', 'location'])

In [313]:
def type_point(lista):
    return {"type": "Point", "coordinates": lista}

In [304]:
coordenadas2 = [144.96262186884945, -37.81227175879301]

In [197]:
#paris_ld = {'type': 'Point', 'coordinates': ['48.8905568192578','2.242512906913018']}

In [222]:
#query = {"location": {"$near": {"$geometry": type_point(coordenadas2), "$minDistance": 0 ,"$maxDistance": 10000}}}

In [314]:
query = {"name": "Airport Bed & Breakfast"}

In [316]:
list(g.find({"name": "Airport Bed & Breakfast"}))

TypeError: 'Cursor' object is not callable

In [317]:
db2 = client.get_database("companies")

In [318]:
db2

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [319]:
g = db2.get_collection("companies")

In [320]:
g.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [321]:
list(g.find({'name': 'Babelgum'}))

TypeError: 'Cursor' object is not callable